# Tutorial: Working with this stactools subpackage

Stactools ([docs](https://stactools.readthedocs.io/en/latest/), [source](https://github.com/stac-utils/stactools)) is a command line tool and library for working with [STAC](https://stacspec.org/), based on [PySTAC](https://github.com/stac-utils/pystac).

[Stactools dataset packages](https://github.com/stactools-packages) are add-ons for stactools that provide STAC functionality for specific datasets, such as [Sentinel 2](https://github.com/stactools-packages/sentinel2) and [Landsat](https://github.com/stactools-packages/landsat).

Stactools and its dataset packages can be accessed from the CLI or from within normal Python code. This notebook provides examples of both.

## 1. Using this notebook

The easiest way to use this notebook is to run it through `scripts/notebook`. This will create a Docker container already set up with the prerequisites and open the stactools dataset folder. Navigate to `docs/` to find this notebook.

If you wish to use this notebook outside of the container (such as on mybinder.org) then please install the prerequisites using:

In [ ]:
!pip install stactools-nclimgrid

## 2. Using the stactools dataset package from the CLI

The first thing we can do is check that the `stac` CLI tool is installed and explore the options. Notice the inclusion of the command for this dataset package in stactools now.

In [17]:
!stac

Usage: stac [OPTIONS] COMMAND [ARGS]...

Options:
  -v, --verbose  Use verbose mode
  -q, --quiet    Use quiet mode (no output)
  --help         Show this message and exit.

Commands:
  add          Add an item to a catalog/collection.
  addraster    Add raster extension to an Item.
  copy         Copy a STAC Catalog
  create-item  Creates an item from an asset
  describe     Prints out a list of all catalogs, collections and items in
               this STAC.
  info         Display info about a static STAC catalog.
  layout       Reformat the layout of a STAC based on templating.
  merge        Merge items from one STAC into another.
  move-assets  Move or copy assets in a STAC to the Item locations.
  nclimgrid    Commands for working with stactools-nclimgrid
  validate     Validate a stac object.
  version      Display version info.


You can now explore the STAC dataset package commands to ingest and describe the data

In [18]:
!stac nclimgrid --help

Usage: stac nclimgrid [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  create-daily-collection    Create a daily NClimGrid STAC collection
  create-daily-item          Create a single daily NClimGrid STAC item
  create-monthly-collection  Create a monthly NClimGrid STAC collection
  create-monthly-item        Create a single monthly NClimGrid STAC item


And more specific help with an individual command.

In [19]:
!stac nclimgrid create-daily-item --help

Usage: stac nclimgrid create-daily-item [OPTIONS] DESTINATION YEAR MONTH DAY
                                        {scaled|prelim} BASE_COG_HREF

  Create a STAC Item for a single day of daily NClimGrid data with optional
  COG creation from NetCDF data.

  DESTINATION (str): A directory where the STAC Item JSON file will be
                     saved
  YEAR (int): STAC Item year
  MONTH (int): STAC Item month
  DAY (int): STAC Item day
  SCALED_OR_PRELIM (str): Choice to use "scaled" or "prelim" data
  BASE_COG_HREF (str): Flat file COG location (COGs are existing or,
                       optionally, created from NetCDF data)

Options:
  --base_nc_href TEXT  option to create COGs from NetCDFs found at this href
  --help               Show this message and exit.


## 3. Using the stactools dataset package from within Python code

So far we've used Jupyter Notebooks' IPython [line magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to work with stactools packages on the command line, but it's also possible to use them within Python scripts.

In [ ]:
import json

from pystac import CatalogType

from stactools.nclimgrid import monthly_stac


# Create a collection with a single item using existing COGS assets. This
# example generates the collection and item in the "examples/monthly" directory.
collection = monthly_stac.create_monthly_collection(
    "189501",
    "189501",
    "../tests/test-data/cog/monthly"
)
collection.catalog_type = CatalogType.RELATIVE_PUBLISHED
collection.set_self_href("temp")
collection.normalize_hrefs("temp")
collection.validate()

print(json.dumps(collection.to_dict(), indent=2))

In [ ]:
import json
from tempfile import TemporaryDirectory

from stactools.nclimgrid import daily_stac


# Create a month of daily items from remote NetCDF data. COGs will also be 
# created. Since data is being downloaded from a remote source, this example may
# take a few minutes to complete.
base_nc_href = "https://www1.ncdc.noaa.gov/pub/data/daily-grids"
with TemporaryDirectory() as tmp_cog_dir:
    month_of_items = daily_stac.create_daily_items(
        2021, 12, "scaled", tmp_cog_dir, base_nc_href=base_nc_href)

print(f"{len(month_of_items)} items were created.\n")
print("The first item:")
print(json.dumps(month_of_items[0].to_dict(), indent=2))